# Task 3: Social Distancing Detector

### Problem Statement: Implement a real time Social Distancing Detector which can identify the distance between two individuals in a crowd.
- Maintained By- **Ajinkya Jadhav**

### Reference :
- Big thanks to **Adrian Rosebrock** explaining how to implement computer vision model practically.🙏🙏
- I have learned to implement this CV model from his tutorial.

### Import required libraries

In [1]:
from scipy.spatial import distance as dist
import numpy as np
import pandas as pd
import imutils
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

In [30]:
# some initial values 
MIN_CONF = 0.5
NMS_THRESH = 0.3
MIN_DISTANCE = 50
personIdx = 0
display = 0

### Project Outline:
- Using `YOLO-v3`pretrained model we can detect whether in video has people or not.
- After that we calculate distance between two individuals using `Euclidean Distance` to check whether they are 2 meters away or not. 
- Those who violate above condition; we count those and show them in red box and those who aren't in green box.

### Function to detect people

In [31]:
def detect_people(frame, net, ln, personIdx = 0):
    # grab the dimensions of the frame and initialize the list of 
    # results
    (H, W) = frame.shape[:2]
    results = []
    
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416),
                                swapRB = True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    # intialize our lists of detected bounding boxes, centroids, and
    # confidences, repectively
    boxes = []
    centroids = []
    confidences = []
    
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e.probablity)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
            # filter detections by ensuring that the object 
            # detected was a person and second is that the minimum
            # confidence is met
            if classID == personIdx and confidence > MIN_CONF:
                # scale the bounding box coordinates relative to 
                # the size of the images, keeping in mind that YOLO
                # actually returns the center (x, y)- cordinates of
                # the bounding box followd by the boxes' width and
                # height
                box = detection[:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype('int')
                
                # use the center (x, y)- cordinates toderive the top
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                
                # update our list of bounding box coordinates,
                # centroids, and confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))
        
    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    # ensure at least one detection exists
    if len(idxs) > 0:

        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # update our results list to consist of the person
            # prediction probability, bounding box coordinates,
            # and the centroid
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    # return the list of results
    return results

In [41]:
# paths of the configuration
base_path = os.getcwd()
config_path = os.path.sep.join([base_path, 'model_config'])
video_path =  os.path.sep.join([base_path, 'Video'])
output_path = os.path.sep.join([base_path, 'Output', 'pedestrians_out.avi'])

In [42]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([config_path, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([config_path, "yolov3.weights"])
configPath = os.path.sep.join([config_path, "yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
print("Loading YOLO from...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

Loading YOLO from...


In [43]:
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
print("Accessing video stream...")
vs = cv2.VideoCapture(os.path.sep.join([video_path, 'pedestrians.mp4']))
writer = None

Accessing video stream...


In [44]:
while True:
    # Read the next frame from the file
    (grabbed, frame) = vs.read()

    # Grab frames from the video stream until we have reached the end frame
    if not grabbed:
        break

    # Resize the frame and then detect people (and only people) in it
    frame = imutils.resize(frame, width=700)
    results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

    # Initialize the set of indexes that violate the minimum social distance
    violate = set()

    # Ensure that there at least two people detections (required in order to
    # compute the pairwise distance maps)
    if len(results) >= 2:
        # Extract all centroids from the results and compute the Euclidean distance between all pairs of centroids
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")

        # Loop over the upper triangular of the distance matrix
        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                # Check to see if the distance between any two centroid pairs is less 
                # than the configured number of pixels

                if D[i, j] < MIN_DISTANCE:
                    # Update the violation set with the indexes of the centroid pairs
                    violate.add(i)
                    violate.add(j)

        # Loop over the results
        for (i, (prob, bbox, centroid)) in enumerate(results):
            # Extract the bounding box and centroid coordinates, then initialize the color of the annotation
            (startX, startY, endX, endY) = bbox
            (cX, cY) = centroid
            color = (0, 255, 0)

            # If the index pair exists within the violation set, then update the color
            if i in violate:
                color = (0, 0, 255)

            # Draw (1) a bounding box around the person and (2) the centroid coordinates 
            # of the person,
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.circle(frame, (cX, cY), 5, color, 1)

        # Draw the total number of social distancing violations on the output frame
        text = "Social Distancing Violations: {}".format(len(violate))
        cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.85, (0, 0, 255), 3)

        # Check to see if the output frame should be displayed in the screen
        if display > 0:
            # Show the output frame
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            # If the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

        # If an output video file path has been supplied and the video writer has not been initialized, do so now
        if display != "" and writer is None:
            # Initialize the video writer
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(output_path, fourcc, 25, (frame.shape[1], frame.shape[0]), True)
            # If the video writer is not None, write the frame to the output video file
        if writer is not None:
            writer.write(frame)